In [ ]:
import cloudscraper
from bs4 import BeautifulSoup as bs
import webbrowser
import time
import random
import json
import os
# 網頁 URL
url = "https://www.mobile01.com/marketcommoditylist.php?c=16&s=20"

# 使用 cloudscraper 來繞過 Cloudflare 的挑戰
headers = {
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"
}
scraper = cloudscraper.create_scraper()
res = scraper.get(url, headers=headers).text
# 使用 BeautifulSoup 解析 HTML
soup = bs(res, "lxml")
# 找到所有分頁按鈕並提取最後一個頁碼
pagination_elements = soup.select('li.l-pagination__page > a.c-pagination[data-page]')
if not pagination_elements:
    raise ValueError("未找到任何分頁按鈕，請檢查選擇器是否正確。")

# 提取最後一個分頁的 data-page 值
last_page_number = int(pagination_elements[-1]["data-page"])
print("最後一個頁碼是:", last_page_number)

def new_page(page_url):
    # 使用正確的選擇器來提取商品名稱和連結
    scraper_page_url = cloudscraper.create_scraper()
    res = scraper_page_url.get(page_url, headers=headers).text
    soup = bs(res, "lxml")
    items = soup.select('div > div.u-gapBottom--lg > a.c-articleCard')
    # print(items)
    for i, item in enumerate(items, 1):

        title = item.text.strip()  # 商品名稱
        link = f"https://www.mobile01.com{item['href']}"  # 完整商品連結
        # print(link)
        open_web(link)




def open_file(items):
    # 檔案名稱
    filename = "M01.json"

    # 如果檔案已存在，載入舊資料
    if os.path.exists(filename):
        with open(filename, "r", encoding="utf-8") as file:
            existing_data = json.load(file)  # 載入現有資料
    else:
        existing_data = []  # 初始化空列表

    # 新增資料到舊資料中
    existing_data.extend(items)

    # 寫入到 JSON 檔案
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(existing_data, file, ensure_ascii=False, indent=4)  # Pretty print JSON
        # print(f"資料已成功儲存到 {filename}")



def open_web(link):
    #print(f"正在打開 {link} 的網頁...")
    # 打开目标网页
    url = link
    # webbrowser.open(link)  # 在預設瀏覽器中打開連結
    scraper_link = cloudscraper.create_scraper()
    res = scraper_link.get(link, headers=headers).text
    soup = bs(res, "lxml")
    elements_title = soup.select_one('div.t2.u-vam > span.u-vam')    
    elements_releasedate = soup.select_one('div.l-input__item.l-input__item--min > span.o-fNotes')
    elements_price = soup.select_one('td.c-detail__info.c-detail__info--middle > span.t2.o-fHighlight')
    elements_content = soup.select_one('div.l-tab > div.l-tab__container.is-active')
    #再寫一欄網址的
    # 輸出結果 
    # 提取每个元素的文本内容并放入列表
    
    title = [element.text.strip() for element in elements_title]
    releasedate = [element.text.strip() for element in elements_releasedate]
    price = [element.text.strip() for element in elements_price]
    content = [element.text.strip() for element in elements_content]
    # 使用正則表達式清理空白行
    cleaned_content = "\n".join(line.strip() for line in content if line.strip())

    # 確保 cleaned_content 是一個列表，且每段內容對應到相應的項目
    cleaned_content_list = cleaned_content.split("\n\n")  # 用雙換行符號分段，形成列表

    # 確認 cleaned_content_list 與其他列表的長度一致
    if len(cleaned_content_list) != len(title):
        raise ValueError("cleaned_content_list 和其他欄位的長度不一致！請檢查資料來源。")
    
    # 使用 zip 將各欄位合併為字典
    items = [
        {"title": t, "releasedate": r, "price": p, "content": c, "link": l} 
        for t, r, p, c, l in zip(title, releasedate, price, cleaned_content_list, [url])
    ]
    open_file(items)
    # # 顯示結果
    # for item in items:
    #     print(f"Title: {item['title']}, Releasedate: {item['releasedate']}, Price: {item['price']}, Content: {item['content']}")
    
 
for i in range (1, last_page_number + 1):
    # params["p"] = i
    page_url = f"https://www.mobile01.com/marketcommoditylist.php?c=16&s=20&p={i}"
    new_page(page_url)
    print(page_url)   
    


